In [ ]:
!pip install svg.path

In [ ]:
import xml.etree.ElementTree as ET # XMLファイルの読み込みと解析を行うためのPython標準ライブラリ。SVGがXML形式なため必要
import json
import math
# svg.pathライブラリはSVGのパスデータをPythonで扱いやすいオブジェクトに翻訳するための外部ライブラリ。
from svg.path import parse_path, Line, CubicBezier, QuadraticBezier
from svg.path.path import Path
import re # 特定の文字列パターン（正規表現）を検索・操作するための標準ライブラリ。svg.pathの補助
from typing import List, Dict, Any
#List:文字列を扱うリスト、Dict:辞書、Any:どんな型でも大丈夫

#SVGファイルからパス情報を取り出し、ストロークデータ（時系列データ）としてndjson形式に変換する。
def svg_to_ndjson_strokes(svg_file_path: str, item_id: str, label: str, style: str) -> Dict[str, Any] | None:

#引数：svg_file_path:SVGファイルへのパス。item_id:ものを識別するためのID。label:アイテムの種類（例: 'Tee'）。style:アイテムのスタイル（例: 'crewneck'）。
#返り値:変換されたndjson形式のデータ（辞書）、エラー時にはNone。

    #(参考: xml.etree.ElementTree)
    try:
        # SVGはXML形式、標準ライブラリのXMLパーサーで読み込む。
        tree = ET.parse(svg_file_path) #SVGファイル全体をツリー構造として読み込む。
        root = tree.getroot() #ツリーの一番上の要素(svgファイルならsvgタグが一番上)を取得しrootに入れる
    except (ET.ParseError, FileNotFoundError):
        print(f"ファイルの読み込みまたは解析エラー")
        return None

    strokes_data = [] #ストロークのリスト。最終的なndjsonのstrokesのキーとして使う。
    stroke_id = 0 #各ストロークにIDをつける。カウンターの役割
    total_time = 0 #ストロークを描くのにかかった時間の合計を記録

    #SVGファイル内の要素は描画された順序で並んでいるのが一般的。この順序を「描画順序」としてそのまま利用する。
    #(参考: SVGの描画モデル)
    #SVGというXMLの規格を使うなら"{http://www.w3.org/2000/svg}"になる
    svg_namespace = "{http://www.w3.org/2000/svg}"

    #SVGのパス要素（path, lineなど）をファイルに書かれた順に取り出しpath_elementに渡す
    #root.findall() はSVGファイルのroot全体から指定された要素を見つける
    #f'.//{svg_namespace}*' はsvg_namespaceに属するすべての要素（*）を見つけよという指示
    for path_element in root.findall(f'.//{svg_namespace}*'):
        # pathタグはd属性に複雑なパス情報を持っている。
        # lineなどの単純なタグはpath形式に変換しておく。(参考: MDNのSVG要素)
        path_string = None #何もない状態にしておく
        if path_element.tag == f'{svg_namespace}path': #pathタグ(ベジェ曲線など複雑目な要素)だったら、
            path_string = path_element.get('d') #そのd属性をpath_string1に渡す
        elif path_element.tag == f'{svg_namespace}line': #lineタグ(直線など)だったら
            #lineタグの情報を<path>タグのMとLコマンドに変換。Mはペンを置く、Lは直線を引く
            x1 = path_element.get('x1', '0')
            y1 = path_element.get('y1', '0')
            x2 = path_element.get('x2', '0')
            y2 = path_element.get('y2', '0')
            path_string = f"M {x1} {y1} L {x2} {y2}"
            #M x1 y1(x1 y1にペンを置く)　L x2 y2(x2 y2までペンを引く)

        if not path_string:
            continue

        try:
            #svg.pathを使ってd属性の文字列をパスオブジェクトに変換する。
            #このライブラリが複雑なベジェ曲線などの計算を代行してくれる。
            #(参考: svg.path)
            path_object = parse_path(path_string)
            #parse_path()がSVGのd属性をPythonで扱えるパスオブジェクトというデータに翻訳してくれる。
        except Exception:
            print(f"パスの解析に失敗")
            continue

        #パスオブジェクトから線をどのくらいの点の集まりに分割するか決める
        points_in_stroke = [] #新しいストロークを一時的に保存するリスト
        path_length = path_object.length() # パスの全長を計算

        # パスの長さに応じて点のサンプリング数をきめる。
        num_points = max(2, int(path_length / 5) + 2) #パスを一定の細かさで分割。要変更。今後考えていく場所。
        #短い線でも始点と終点の2点は確保。

        for i in range(num_points):
            #t=0が始点t=1が終点とする(参考: svg.path)
            t = i / (num_points - 1) #パスがどの位置にあるかを示す。(t:線全体の描画を0～1とする)
            point = path_object.point(t) #tの位置を渡すだけでそこのx、y座標を出してくれる(point()ライブラリ)
            x, y = point.real, point.imag
            #realで実数部分、imagで虚数部分。svg.pathは点を複素数で扱うためxy座標に変換

            #パスの進行度(t)と長さ(path_length)から描画時間を計算する。
            #(t:線全体の描画を0～1とする)。×5で調整。変更可。パスの長さ1単位につき5単位の時間がかかる
            simulated_time = int(total_time + (t * path_length * 5))
            #total_tima、それまでに描いたストロークにかかった時間の合計

            #xy座標と描画時間を一つの辞書にまとめ、ストロークのリストに追加
            points_in_stroke.append({"x": x, "y": y, "t": simulated_time})

        stroke_id += 1 #すつろーくに何番目の線かの番号をつける
        #ストローク番号と点のリストを組み合わせ、全体のリストに追加する。
        strokes_data.append({
            "stroke_id": stroke_id,
            "points": points_in_stroke
        })

        # 次のストロークのために前回のストロークの時間を合計時間に加算する。
        # ->前のストロークの終了時間から始められる。
        # 全体の描画が連続した時系列データとして扱える
        total_time += int(path_length * 5)

    ndjson_object = {
        "item_id": item_id,
        "label": label,
        "style": style,
        "strokes": strokes_data
    }

    return ndjson_object

#実行
if __name__ == "__main__":
    svg_file = 'Tee 1.svg'

    ndjson_data = svg_to_ndjson_strokes(svg_file, "Tee 1", "Tee", "crewneck")

    if ndjson_data:
        # 結果をndjsonファイルに
        output_file = 'apparel_dataset.ndjson'
        with open(output_file, 'a', encoding='utf-8') as f:
            json.dump(ndjson_data, f, ensure_ascii=False)
            f.write('\n')

        print(f"SVGからndjsonへの変換が完了: {output_file}")